<a href="https://colab.research.google.com/github/ganiket25201001/awp/blob/main/test_WAF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
from IPython.display import display
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
import joblib

In [15]:
#data = pd.read_csv("",index_col="index")
data = pd.read_csv("final-allpayload.csv",index_col="index")

In [16]:
data

,payload,is_malicious,injection_type,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
index,,,,,,,,,,,,,
0,37662577P,0.0,LEGAL,9,0,0,50,80,56.333333,8.537499,6,0,0
1,shirting,0.0,LEGAL,8,0,0,103,116,109.000000,5.049752,7,0,1
2,&kw=%27;alert%28%27XSS%27%29;//,1.0,XSS,31,0,11,37,119,65.806452,26.617263,18,0,1
3,obeying,0.0,LEGAL,7,0,0,98,121,107.000000,7.151423,7,0,1
4,dictating,0.0,LEGAL,9,0,0,97,116,105.666667,6.599663,7,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110355,needlessness,0.0,LEGAL,12,0,0,100,115,107.666667,6.209312,5,1,0
110356,Mary-Jo,0.0,LEGAL,7,0,1,45,121,91.285714,25.257046,7,0,0
110357,marshaled,0.0,LEGAL,9,0,0,97,115,105.000000,6.463573,8,0,0


In [17]:
Y = data['is_malicious']
X = data.iloc[:,3:]
X

,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
index,,,,,,,,,,
0,9,0,0,50,80,56.333333,8.537499,6,0,0
1,8,0,0,103,116,109.000000,5.049752,7,0,1
2,31,0,11,37,119,65.806452,26.617263,18,0,1
3,7,0,0,98,121,107.000000,7.151423,7,0,1
4,9,0,0,97,116,105.666667,6.599663,7,0,1
...,...,...,...,...,...,...,...,...,...,...
110355,12,0,0,100,115,107.666667,6.209312,5,1,0
110356,7,0,1,45,121,91.285714,25.257046,7,0,0
110357,9,0,0,97,115,105.000000,6.463573,8,0,0


In [18]:
import xgboost as xgb
#?xgb.XGBClassifier()
xgb_classifer = xgb.XGBClassifier()
xgb_classifer.fit(X,Y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [19]:
joblib.dump(xgb_classifer, 'xgb_classifier.pkl')

['xgb_classifier.pkl']

In [20]:
data['predicted_is_malicious'] = xgb_classifer.predict(X)

data.head(30)

data[["is_malicious","predicted_is_malicious"]]


,is_malicious,predicted_is_malicious
index,,
0,0.0,0
1,0.0,0
2,1.0,1
3,0.0,0
4,0.0,0
...,...,...
110355,0.0,0
110356,0.0,0
110357,0.0,0


In [21]:
independent_variables=['length', 'non-printable','punctuation', 'min-byte', 'max-byte', 'mean-byte', 'std-byte','distinct-byte', 'sql-keywords', 'js-keywords']
independent_variables

independent_variables=data.columns

In [22]:
sql_keywords = pd.read_csv('/content/SQLKeywords.txt', index_col=False)
js_keywords = pd.read_csv("/content/JavascriptKeywords.txt",index_col=False)

In [23]:
def calculate_features_and_predict(payload):
  features = {}
  payload = str(payload)
  features['length'] = len(payload)
  features['non-printable'] = len([1 for letter in payload if letter not in string.printable])
  features['punctuation'] = len([1 for letter in payload if letter in string.punctuation])
  features['min-byte'] = min(bytearray(payload,'utf-8'))
  features['max-byte'] = max(bytearray(payload,'utf-8'))
  features['mean-byte'] = np.mean(bytearray(payload,'utf-8'))
  features['std-byte'] = np.std(bytearray(payload,'utf-8'))
  features['distinct-byte'] = len(set(bytearray(payload,'utf-8')))
  features['sql-keywords'] = len([1 for keyword in sql_keywords['Keyword'] if str(keyword).lower() in payload.lower()])
  features['js-keywords'] = len([1 for keyword in js_keywords['Keyword'] if str(keyword).lower() in payload.lower()])
  #payload_df = pd.DataFrame(data=features,index=[0],columns=independent_variables)
  payload_df = pd.DataFrame(features,index=[0])
  display(payload_df)
  result = xgb_classifer.predict(payload_df)
  #display(result)
  return result[0]


In [24]:
calculate_features_and_predict("<script>")

,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
0,8,0,2,60,116,97.875,21.951296,8,0,0


np.int64(1)

In [ ]:
payload=''
while (payload != 'exit' ):
  payload = input("Enter payload")
  result = calculate_features_and_predict(payload)
  if(result > 0):
    print(f"Your payload {payload} is malicious - 403 error\n")
  else:
    print(f"Your payload {payload} is safe 200 OK\n")

Enter payloadhii


,length,non-printable,punctuation,min-byte,max-byte,mean-byte,std-byte,distinct-byte,sql-keywords,js-keywords
0,3,0,0,104,105,104.666667,0.471405,2,0,0


Your payload hii is safe 200 OK

